# Pixyzの確率分布の記述方法

ここではまず，Pixyzにおける確率モデルの実装方法について説明します.  
Distribution API document: https://docs.pixyz.io/en/latest/distributions.html

In [1]:
from __future__ import print_function
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np

torch.manual_seed(1)

In [2]:
from pixyz.utils import print_latex

## 1. 深層ニューラルネットワークを用いない確率分布の定義

### 1.1 シンプルな確率分布の定義

ガウス分布を作るためには，`Normal`をインポートして，平均（loc）と標準偏差（scale）を定義します．

In [3]:
from pixyz.distributions import Normal

x_dim = 50
p1_nor_x = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.), var=['x'], features_shape=[x_dim], name='p_{1}')

なお``var``には，変数の名前を設定します．ここでは`"x"`を設定しています．

また，features_shapeでは次元数を指定します．ここではfeatures_shapeが50となっていますから，50次元のサンプルを生成する形になります．

上記で定義したp1の情報は次のようにみることができます．

In [4]:
print(p1_nor_x.distribution_name) 
print(p1_nor_x.prob_text)

Normal
p_{1}(x)


distribution_nameでは，確率分布の名前を確認できます．

prob_textでは，確率分布の形をテキストで出力できます．ここでテキストに書かれている確率変数は，上記のvarで指定したものです.

また，p1を丸ごとprintすると，以下のように表示されます．

In [5]:
print(p1_nor_x)

Distribution:
  p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )


print_latexを利用するとLaTex表記で定義した確率分布が表示されます  
注: 数式のtex形式への出力に外部ライブラリのsympy使用しており，sympyの影響で数式の結果に支障を与えないが，数式の順序が入れ替わることがある  
print_latex(A +B)の出力が  
B+Aになることがあったりする

In [6]:
print_latex(p1_nor_x)

<IPython.core.display.Math object>

次に，定義した分布からサンプリングしてみましょう． サンプリングは，`sample()`によって実行します．

In [7]:
p1_nor_x_samples = p1_nor_x.sample()
print(p1_nor_x_samples)
print('--------------------------------------------------------------------------')
print(p1_nor_x_samples["x"])

{'x': tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798, -1.6091,
         -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,  0.2284,  0.4676,
         -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073,
          1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177, -0.7981, -0.1316,
          1.8793, -0.0721,  0.0663, -0.4370,  0.7626,  0.4415,  1.1651,  2.0154,
          0.2152, -0.5242, -0.1860, -0.6446,  1.5392, -0.8696, -3.3312, -0.7479,
          1.1173,  0.2981]])}
--------------------------------------------------------------------------
tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798, -1.6091,
         -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,  0.2284,  0.4676,
         -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073,
          1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177, -0.7981, -0.1316,
          1.8793, -0.0721,  0.0663, -0.4370,  0.7626,  0.4415,  1.1651,  2.0154

出力はdict形式になっています．

サンプリング結果を確認したい変数について指定することで，中身を確認できます（ただし，この例では変数は"x"のみです）．

なお，サンプリング結果は，PyTorchのtensor形式になっています．

### 1.2 条件付確率分布の定義

次に条件付確率分布の定義の仕方を正規分布の例で見ていきます

正規分布ではパラメータは平均$\mu$と分散$\sigma^2$がありますが，今回は平均が条件付けられた正規分布を取り上げたいと思います

$p(x|\mu_{var}) = \cal N(x; \mu=\mu_{var}, \sigma^2=1)$

分布の条件付き変数の設定はcond_varで行います  
ここではmu_varという変数を正規分布の平均に設定したいため  
cond_var=['mu_var']  
loc='mu_var'  
とします

In [8]:
x_dim = 50
p1_nor_x__mu = Normal(loc='mu_var', scale=torch.tensor(1.), var=['x'], cond_var=['mu_var'], features_shape=[x_dim])

In [9]:
print(p1_nor_x__mu)
print_latex(p1_nor_x__mu)

Distribution:
  p(x|\mu_{var})
Network architecture:
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['mu_var'], input_var=['mu_var'], features_shape=torch.Size([50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

これで平均が$\mu_{var}$で条件付けされる正規分布が定義できました  
試しに$\mu_{var}=0$としてxをサンプリングしてみます  
sampleメソッドにdict形式で変数を指定します

In [10]:
p1_nor_x__mu.sample({"mu_var": 0})

{'mu_var': 0,
 'x': tensor([[-0.5962, -1.0055, -0.2106, -0.0075,  1.6734,  0.0103,  0.9837,  0.8793,
          -0.9962, -0.8313, -0.4610, -0.5601,  0.3956, -0.9823,  1.3264,  0.8547,
          -0.6540,  0.7317, -1.4344, -0.5008,  0.1716, -0.1600, -0.5047, -1.4746,
          -1.0412,  0.7323, -1.0483, -0.4709,  0.2911,  1.9907, -0.9247, -0.9301,
           0.8165, -0.9135,  0.2053,  0.3051,  0.5357, -0.4312,  0.1573,  1.2540,
           1.3275, -0.4954, -1.9804,  1.7986,  0.1018,  0.3400, -0.6447, -0.2870,
           3.3212, -0.4021]])}

次に$\mu_{var}$自体も何らかの確率分布に従う変数とし，その確率分布を定めます  
ここでは仮にベルヌーイ分布とします  
$p(\mu_{var}) = \cal B(\mu_{var};p=0.3)$

In [11]:
from pixyz.distributions import Bernoulli
p2_ber_mu = Bernoulli(probs=torch.tensor(0.3), var=['mu_var'], features_shape=[x_dim])

In [12]:
print(p2_ber_mu)
print(p2_ber_mu.sample())
print_latex(p2_ber_mu)

Distribution:
  p(\mu_{var})
Network architecture:
  Bernoulli(
    name=p, distribution_name=Bernoulli,
    var=['mu_var'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (probs): torch.Size([1, 50])
  )
{'mu_var': tensor([[0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
         1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])}


<IPython.core.display.Math object>

Pixyzでは分布の積は，掛け算で表すことができます  
定義した$p(\mu_{var})$と$p(x|\mu_{var})$を掛け合わせて同時分布$p(x, \mu_{var})$を定義します  
$p(x, \mu_{var}) = p(x|\mu_{var}) p(\mu_{var})$


In [13]:
p_joint_mu_x = p1_nor_x__mu * p2_ber_mu
print(p_joint_mu_x) 
print_latex(p_joint_mu_x)

Distribution:
  p(x,\mu_{var}) = p(x|\mu_{var})p(\mu_{var})
Network architecture:
  Bernoulli(
    name=p, distribution_name=Bernoulli,
    var=['mu_var'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (probs): torch.Size([1, 50])
  )
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['mu_var'], input_var=['mu_var'], features_shape=torch.Size([50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

同時分布でも今までと同様にsampleメソッドでサンプリングを行うことができます  
全ての変数とその値がdict形式で出力されます

In [14]:
p_joint_mu_x.sample()

{'mu_var': tensor([[1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
          0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.]]),
 'x': tensor([[ 3.6415, -0.9624,  0.7924, -1.3889,  1.0127, -0.8734,  1.7997,  1.2824,
           1.6604,  0.2717,  0.1913,  0.1267,  0.5707,  0.8652,  0.3437,  0.3718,
           0.1444,  1.7772, -2.3381,  0.1709,  1.1661,  1.4787,  0.2676,  0.7561,
          -0.5873, -2.0619,  0.4305,  0.3377, -0.3438, -0.6172,  2.2530, -0.0514,
          -1.0257,  0.5213, -2.3065,  1.6037,  0.1794,  0.1447,  0.6411,  0.4793,
           0.7617, -0.3542, -0.2693,  2.3120, -0.8920, -0.7529, -0.0573,  2.2000,
           0.9912,  0.9414]])}

## 2. 深層ニューラルネットワークと組み合わせた確率分布の設定

次に， 確率分布のパラメータを深層ニューラルネットワークで定義します．

例えば，ガウス分布の平均$\mu$と分散$\sigma^2$は， パラメータ$\theta$を持つ深層ニューラルネットワークによって，$\mu=f(x;\theta)$および$\sigma^2=g(x;\theta)$と定義できます．

したがって，ガウス分布は${\cal N}(\mu=f(x;\theta),\sigma^2=g(x;\theta))$となります．

$p(a) = {\cal N}(a; \mu=f(x;\theta),\sigma^2=g(x;\theta))$を定義してみましょう

Pixyzでは，次のようなクラスを記述することで，これを実現できます．

In [15]:
a_dim = 20

class ProbNorAgivX(Normal):
    """
    Probability distrituion Normal A given X
    p(a) = {\cal N}(a; \mu=f(x;\theta),\sigma^2=g(x;\theta)
    loc and scale are parameterized by theta given x
    """
    def __init__(self):
        super(ProbNorAgivX, self).__init__(cond_var=['x'], var=['a'])
        
        self.fc1 = nn.Linear(x_dim, 10)
        self.fc_loc = nn.Linear(10, a_dim)
        self.fc_scale = nn.Linear(10, a_dim)
        
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        return {'loc': self.fc_loc(h1), 'scale': F.softplus(self.fc_scale(h1))}
p_nor_a__x = ProbNorAgivX()

まず， ガウス分布クラスを継承することで，ガウス分布のパラメータを深層ニューラルネットワークで定義することを明示します．

次に，コンストラクタで，利用するニューラルネットワークを記述します．これは，通常のPyTorchと同じです．

唯一異なる点は，superの引数にvarとcond_varの名前を指定している点です．

varは先程見たように，出力する変数の名前を指定します．一方，cond_varではニューラルネットワークの入力変数の名前を指定します．これは，ここで定義する分布において，条件付けられる変数とみなすことができます．

forwardについても，通常のPyTorchと同じです．ただし，注意点が2つあります．

* 引数の名前と数は，cond_varで設定したものと同じにしてください． 例えば，cond_var=["x", "y"]とした場合は，forward(self, x, y)としてください．
* 戻り値は，それぞれの確率分布のパラメータになります．上記の例ではガウス分布なので，平均と分散を指定しています．

そして最後に，定義した確率分布クラスのインスタンスを作成します．

次に，先程の例と同様，確率分布の情報を見てみましょう.

In [16]:
print(p_nor_a__x)
print_latex(p_nor_a__x)

Distribution:
  p(a|x)
Network architecture:
  ProbNorAgivX(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc_loc): Linear(in_features=10, out_features=20, bias=True)
    (fc_scale): Linear(in_features=10, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

p2の分布は，xで条件付けた形になっています．これらの表記は，superの引数で設定したとおりになっています．

次に，先程の例のように，サンプリングしてみましょう．

注意しなければならないのは，先ほどと異なり，条件づけた変数xがあるということです．

x_samplesをxとしてサンプリングしましょう．

In [17]:
x_samples = torch.Tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085,
         -0.2155,  2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196,
         -0.5484, -0.3472,  0.4730, -0.4286,  0.5514, -1.5474,  0.7575,
         -0.4068, -0.1277,  0.2804,  1.7460,  1.8550, -0.7064,  2.5571,
          0.7705, -1.0739, -0.2015, -0.5603, -0.6240, -0.9773, -0.1637,
         -0.3582, -0.0594, -2.4919,  0.2423,  0.2883, -0.1095,  0.3126,
         -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880, -1.1435,
         -0.6512]])

In [18]:
p_nor_a__x_samples = p_nor_a__x.sample({'x': x_samples})
print(p_nor_a__x_samples)
print(p_nor_a__x_samples['a'])
print(p_nor_a__x_samples['x'])

{'x': tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085, -0.2155,
          2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196, -0.5484, -0.3472,
          0.4730, -0.4286,  0.5514, -1.5474,  0.7575, -0.4068, -0.1277,  0.2804,
          1.7460,  1.8550, -0.7064,  2.5571,  0.7705, -1.0739, -0.2015, -0.5603,
         -0.6240, -0.9773, -0.1637, -0.3582, -0.0594, -2.4919,  0.2423,  0.2883,
         -0.1095,  0.3126, -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880,
         -1.1435, -0.6512]]), 'a': tensor([[-1.7231e-01, -5.0856e-01,  1.3573e+00, -7.1246e-01,  3.8644e-01,
          1.1225e+00,  1.4864e-01,  6.8819e-02, -5.6884e-01, -2.4427e+00,
          1.2279e-03, -9.0337e-01,  5.3217e-02,  6.0509e-01, -3.8033e-01,
          6.5707e-02, -2.3049e-01,  3.4607e-01,  2.6745e-02, -3.9659e-01]])}
tensor([[-1.7231e-01, -5.0856e-01,  1.3573e+00, -7.1246e-01,  3.8644e-01,
          1.1225e+00,  1.4864e-01,  6.8819e-02, -5.6884e-01, -2.4427e+00,
          1.2279e-03, -9.0

出力には，aとxの２つのサンプルがあります．

aが今回計算したサンプルで，xについては，引数として与えたサンプルがそのまま入っています．

### Next Tutorial
02-LossAPITutorial.ipynb